In [1]:
import ipywidgets as widgets
from astropy.table.jsviewer import JSViewer
from astropy.table import conf

class FloatingAstropyTable():
    instance = None

    def __init__(self, ipython,table):
        """Public constructor."""
        if FloatingAstropyTable.instance is not None:
            raise Exception("""Only one instance of the Floating AstropyTable can exist at a 
            time.  Call close() on the active instance before creating a new instance.
            If you have lost the handle to the active instance, you can re-obtain it
            via `FloatingAstropyTable.instance`.""")

        FloatingAstropyTable.instance = self
        self.closed = False
        self.table = table
        #self.namespace = NamespaceMagics()
        #self.namespace.shell = ipython.kernel.shell

        self._box = widgets.Box()
        self._box._dom_classes = ['inspector']
        self._box.background_color = '#fff'
        self._box.border_color = '#ccc'
        self._box.border_width = 1
        self._box.border_radius = 5

        self._modal_body = widgets.VBox()
        self._modal_body.overflow_y = 'scroll'

        self._modal_body_label = widgets.HTML(value = 'Not hooked')
        self._modal_body.children = [self._modal_body_label]

        self._box.children = [
            self._modal_body, 
        ]

        self._ipython = ipython
        self._ipython.events.register('post_run_cell', self._fill)

    def close(self):
        """Close and remove hooks."""
        if not self.closed:
            self._ipython.events.unregister('post_run_cell', self._fill)
            self._box.close()
            self.closed = True
            VariableInspectorWindow.instance = None

    def _fill(self):
        """Fill self with variable information."""
        #values = self.namespace.who_ls()
        
        tableid = 'table{0}-{1}'.format(id(self),
                                            np.random.randint(1, 1e6))
        jsv = JSViewer(display_length=10)
        table_class = conf.default_notebook_table_class
        html = self.table._base_repr_(html=True, max_width=-1, tableid=tableid,
                                         max_lines=-1, show_dtype=False,
                                         tableclass=table_class)

        html += jsv.ipynb(tableid, css=None)


        self._modal_body_label.value = html
        
    def _ipython_display_(self):
        """Called when display() or pyout is used to display the Variable 
        Inspector."""
        self._box._ipython_display_()

In [2]:
import astropy.table as tab
a=tab.Table()

In [3]:
import numpy as np
a['donuts']=np.random.rand(100)

In [4]:
a['dina']=np.random.rand(100)

In [5]:
inspector=FloatingAstropyTable(get_ipython(),a)

In [6]:
inspector

In [7]:
%%javascript
$('div.inspector')
    .detach()
    .prependTo($('body'))
    .css({
        'z-index': 999, 
        position: 'fixed',
        'box-shadow': '5px 5px 12px -3px black',
        opacity: 0.9
    })
    .draggable();

<IPython.core.display.Javascript object>